# Install

In [ ]:
!pip3 install soynlp

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [ ]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/ltokenizer/')
print(sys.path)

In [ ]:
import re
import swifter
import config as cfg
from pshmodule.utils import filemanager as fm

In [ ]:
df = fm.load(cfg.origin_data_colab)

extension : .pickle
Loaded 10317 records from drive/MyDrive/MemeProject//data/temp_for_doc2vec.pickle


In [ ]:
df['content'] = df.content.swifter.apply(lambda x: re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', x))

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
print(f"df : {df.head(10)}")

df :                   content                       pos
0         나 중간고사 반에서 1등했어              [나, 중간고사, 반]
1  중간고사 점수 내가 반에서 제일 잘 받음  [중간고사, 점수, 내, 반, 제일, 받음]
2       나 반에서 중간 성적 제일 좋아    [나, 반, 중간, 성적, 제일, 좋아]
3    우리 반에서 내가 시험 제일 잘 봤다    [우리, 반, 내, 시험, 제일, 봤다]
4            중간고사 반 1등 먹음             [중간고사, 반, 먹음]
5          중간시험 내가 반 1등이야            [중간, 시험, 내, 반]
6            너 왜 이렇게 멍청하냐         [너, 왜, 이렇게, 멍청하냐]
7             이 새끼 빡대가리잖아          [새끼, 빡, 대가리, 잖아]
8            너 말귀 못 알아듣냐고         [너, 말귀, 못, 알아듣냐고]
9              병신 이해력 딸리네            [병신, 이해력, 딸리네]


# Soynlp

##### WordExtractor
새로운 단어가 나올 때 미등록 단어 문제가 발생하는 것을 해결하기 위한 통계기반 단어 학습 기법

In [ ]:
from soynlp.word import WordExtractor

In [ ]:
word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

In [ ]:
sentence = df.content.tolist()
sentence[:10]

['나 중간고사 반에서 1등했어',
 '중간고사 점수 내가 반에서 제일 잘 받음',
 '나 반에서 중간 성적 제일 좋아',
 '우리 반에서 내가 시험 제일 잘 봤다',
 '중간고사 반 1등 먹음',
 '중간시험 내가 반 1등이야',
 '너 왜 이렇게 멍청하냐',
 '이 새끼 빡대가리잖아',
 '너 말귀 못 알아듣냐고',
 '병신 이해력 딸리네']

In [35]:
len(sentence)

10317

In [ ]:
word_extractor.train(sentence)  # list of str or like
words = word_extractor.extract()

training was done. used memory 0.329 Gb
all cohesion probabilities was computed. # words = 1856
all branching entropies was computed # words = 11743
all accessor variety was computed # words = 11743


In [ ]:
words['중간고사']

Scores(cohesion_forward=0.5598113698142824, cohesion_backward=0, left_branching_entropy=1.1972375015579317, right_branching_entropy=1.564957250242801, left_accessor_variety=4, right_accessor_variety=5, leftside_frequency=20, rightside_frequency=0)

In [ ]:
words

##### Noun Extractor
soynlp 명사 추출기

In [ ]:
# from soynlp.noun import LRNounExtractor_v2 #noun extracter v2

In [ ]:
# noun_extractor = LRNounExtractor_v2(verbose=True)
# nouns = noun_extractor.train_extract(sentence)

In [ ]:
# print(list((nouns.keys()))[:150])

##### LTokenizer

In [ ]:
import swifter

In [ ]:
from soynlp.tokenizer import LTokenizer

In [ ]:
cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

# key, value로 사용자 사전 넣기 1.0
tokenizer = LTokenizer(scores=cohesion_score)

In [ ]:
sent = "내가 중간고사에서 점수 제일 잘 받았다."

In [ ]:
print(tokenizer.tokenize(sent))

['내가', '중간고사', '에서', '점수', '제일', '잘', '받았', '다.']


In [ ]:
df['soynlp'] = df.content.swifter.apply(lambda x: tokenizer.tokenize(x))

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
df.head(20)

,content,pos,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]","[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]","[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]","[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]","[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]","[중간고사, 반, 1등, 먹음]"
5,중간시험 내가 반 1등이야,"[중간, 시험, 내, 반]","[중간, 시험, 내가, 반, 1등, 이야]"
6,너 왜 이렇게 멍청하냐,"[너, 왜, 이렇게, 멍청하냐]","[너, 왜, 이렇게, 멍청하냐]"
7,이 새끼 빡대가리잖아,"[새끼, 빡, 대가리, 잖아]","[이, 새끼, 빡대가리잖아]"
8,너 말귀 못 알아듣냐고,"[너, 말귀, 못, 알아듣냐고]","[너, 말귀, 못, 알아, 듣냐고]"
9,병신 이해력 딸리네,"[병신, 이해력, 딸리네]","[병신, 이해력, 딸리네]"
